[View in Colaboratory](https://colab.research.google.com/github/LuizFelipeLemon/machine_learning/blob/master/svm_multclass.ipynb)

In [0]:
# Support Vector Machine (SVM)
# from https://www.superdatascience.com/machine-learning/ 


# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics as m

In [0]:
import os
import tarfile
from six.moves import urllib

FILE_TO_DOWNLOAD =  "bla.csv"
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/GabrielVantuil/ECT2702-TAI/"
DATA_PATH = "master/"
DATA_URL = DOWNLOAD_ROOT + DATA_PATH + FILE_TO_DOWNLOAD

def fetch_data(data_url=DATA_URL, data_path=DATA_PATH, file_to_download=FILE_TO_DOWNLOAD):
  if not os.path.isdir(data_path):
    os.makedirs(data_path)
  urllib.request.urlretrieve(data_url, data_path+file_to_download)
  
  
fetch_data()

# observando se o diretório datasets foi criado com sucesso 

In [20]:
# Importing the dataset
dataset = pd.read_csv(DATA_PATH+FILE_TO_DOWNLOAD)


dataset.head(  )

,R,G,B,cor
0,16,47,33,0
1,16,48,34,0
2,16,47,34,0
3,16,40,34,0
4,16,47,33,0


In [0]:
X = dataset.iloc[:,:3].values
y = dataset.iloc[:,3].values



In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [23]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [0]:
# Fitting SVM to the Training set
#from sklearn.svm import SVC
#classifier = SVC(kernel = 'linear', random_state = 0)
#classifier = SVC(decision_function_shape='ovo')

#classifier.fit(X_train, y_train)

In [41]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
#classifier = SVC(kernel = 'linear', random_state = 0)
classifier = SVC(decision_function_shape='ovo')

classifier.fit(X_train, y_train)

C_2d_range = np.logspace(-2,10,30)

gamma_2d_range = np.logspace(-9,3,30)



print(C_2d_range)
classifiers = []


for C in C_2d_range:
    for gamma in gamma_2d_range:
      
        classifier = SVC(C=C, gamma=gamma)
        classifier.fit(X_train, y_train)
        
        y_pred = classifier.predict(X_test)
        
        error = m.mean_squared_error(y_pred,y_test)
        
        
        classifiers.append((C, gamma, error,y_pred))        
       
print(len(classifiers))


[1.00000000e-02 2.59294380e-02 6.72335754e-02 1.74332882e-01
 4.52035366e-01 1.17210230e+00 3.03919538e+00 7.88046282e+00
 2.04335972e+01 5.29831691e+01 1.37382380e+02 3.56224789e+02
 9.23670857e+02 2.39502662e+03 6.21016942e+03 1.61026203e+04
 4.17531894e+04 1.08263673e+05 2.80721620e+05 7.27895384e+05
 1.88739182e+06 4.89390092e+06 1.26896100e+07 3.29034456e+07
 8.53167852e+07 2.21221629e+08 5.73615251e+08 1.48735211e+09
 3.85662042e+09 1.00000000e+10]
900


In [40]:
classifiers.sort(key = lambda x:x[2], reverse = False)
print(classifiers[0][:3])

(0.06723357536499334, 1.2689610031679235, 0.008)


In [36]:
# Predicting the Test set results
y_pred =classifiers[0][3]

print(y_test[0:35])
print(y_pred[0:35])

[0 2 2 4 4 0 3 4 1 1 2 2 3 2 1 3 4 3 1 0 2 3 2 1 0 4 0 3 0 2 4 1 1 1 3]
[0 2 2 4 4 0 3 4 1 1 2 2 3 2 1 3 4 3 1 0 2 3 2 1 0 4 0 3 0 2 4 2 1 1 3]


In [37]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

[[22  0  0  0  0]
 [ 0 24  1  0  0]
 [ 0  0 26  0  0]
 [ 0  0  0 28  0]
 [ 0  0  0  0 24]]


In [38]:
# Visualising the Training set results
from matplotlib.colors import ListedColormap
X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('blue', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('SVM (Training set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

ValueError: ignored